In [1]:
import pandas as pd
path = '../../../results/information_bottleneck/evaluation/qwen_1.7B/en/eval_cohere_r_command_plus_qwen1.7b.csv'
df = pd.read_csv(path)

In [2]:
df

,Unnamed: 0,index,question,options,gold,explanation,pred_noexp,prob_noexp,pred_exp,prob_exp,is_correct_noexp,is_correct_exp,sufficiency,probs_noexp,probs_exp,ppl_qwen
0,0,0,George wants to warm his hands quickly by rubb...,"['dry palms', 'wet palms', 'palms covered with...",A,Dry palms will produce the most heat when rubb...,B,0.547882,A,0.994844,False,True,True,[0.42669106 0.54788214 0.00418313 0.0212437 ],[9.9484384e-01 2.7705361e-03 4.8144700e-04 1.9...,19.963142
1,1,1,Which of the following statements best explain...,"['The refrigerator door is smooth.', 'The refr...",B,Refrigerator doors are often made of materials...,B,0.887797,B,0.991470,True,True,True,[0.05675495 0.88779676 0.00536217 0.05008607],[0.00588399 0.9914697 0.00115863 0.00148771],9.792212
2,2,2,A fold observed in layers of sedimentary rock ...,"['cooling of flowing magma.', 'converging of c...",B,The convergence of crustal plates can cause in...,B,0.756387,B,0.992008,True,True,True,[0.16844344 0.756387 0.03758481 0.03758481],[0.00593047 0.9920083 0.00103056 0.00103056],7.674981
3,3,3,Which of these do scientists offer as the most...,"['worldwide disease', 'global mountain buildin...",D,The impact of an asteroid creating dust that b...,D,0.814510,D,0.850181,True,True,True,[0.15944956 0.02445238 0.00158781 0.8145103 ],[1.4802805e-01 1.0617323e-03 7.2971720e-04 8.5...,7.413361
4,4,4,A boat is acted on by a river current flowing ...,"['west', 'east', 'north', 'south']",B,"If the boat is traveling northeast, it means i...",B,0.745938,A,0.900730,True,False,True,[0.13064119 0.7459377 0.06171053 0.06171053],[0.9007304 0.0836992 0.00778524 0.00778524],11.937057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2576,2576,2576,Which of these processes involves the transfer...,"['erosion', 'sedimentation', 'subduction', 'ce...",C,Subduction is a geological process where an oc...,C,0.916285,C,0.995289,True,True,True,[0.07499321 0.00256613 0.91628486 0.00615582],[4.1315737e-03 8.5748077e-05 9.9528921e-01 4.9...,8.024777
2577,2577,2577,"In a forest, how do decomposers help other org...",['They release oxygen into the air that animal...,B,Decomposers break down dead organic matter and...,B,0.974788,B,0.988325,True,True,True,[0.02023107 0.9747877 0.00146553 0.00351563],[9.6290661e-03 9.8832524e-01 8.9564268e-04 1.1...,4.555921
2578,2578,2578,What is the best way to conserve natural resou...,"['Throw all glass in the trash.', 'Use paper t...",C,Conserving water is an important aspect of pre...,C,0.964807,C,0.994786,True,True,True,[0.02890791 0.00237291 0.96480685 0.00391226],[4.0654629e-03 7.0647150e-04 9.9478596e-01 4.4...,10.305088
2579,2579,2579,Which describes the composition of carbohydrates?,"['lipids bonding to form phospholipids', 'mono...",D,"Carbohydrates are composed of saccharides, whi...",B,0.874842,D,0.667682,False,True,True,[2.0534171e-02 8.7484217e-01 3.4243971e-04 1.0...,[0.01567173 0.31477502 0.00187172 0.6676815 ],7.238341


In [16]:
import pandas as pd
import logging
from tqdm import tqdm
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
import sys

# === 1. Setup Logging ===
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("agent_conciseness.log", encoding='utf-8'),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)
logger.info("Starting LangChain Agent Evaluation...")

# === 3. Sample 10 examples with low sufficiency ===
sample_df = df[df["sufficiency"] == False].sample(10, random_state=42)
logger.info(f"Sampled {len(sample_df)} low-sufficiency examples")

# === 4. Configure OpenRouter LLM ===
llm = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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",
    model="openai/gpt-4o-mini",
    temperature=0.3
)
# === 5. Prompt Template ===
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an explanation analyst. Your task is to identify and remove redundancy."),
    ("user", """You are given:
- A multiple-choice question
- The correct answer
- An explanation justifying that answer

Your tasks:
1. Identify any redundant or verbose parts in the explanation.
2. Justify why they are unnecessary.
3. Rewrite the explanation concisely while keeping its meaning and justification.

Return:
---
Question: {question}
Answer: {answer}
Redundant parts:
- ...
Reasons:
- ...
Concise Explanation:
...

Explanation:
\"\"\"{explanation}\"\"\"
""")
])

# === 6. LangChain Chain ===
agent_chain = LLMChain(llm=llm, prompt=prompt)

# === 7. Run Agent
results = []
for row in tqdm(sample_df.itertuples(), total=len(sample_df)):
    try:
        input_data = {
            "question": row.question,
            "answer": row.gold,
            "explanation": row.explanation
        }
        logger.info(f"Running agent on index: {row.index}")
        agent_output = agent_chain.run(input_data)

        results.append({
            "index": row.index,
            "question": row.question,
            "gold": row.gold,
            "explanation": row.explanation,
            "sufficiency": row.sufficiency,
            "is_correct_exp": row.is_correct_exp,
            "ppl_qwen": row.ppl_qwen,
            "agent_output": agent_output
        })

    except Exception as e:
        logger.error(f"Error on index {row.index}: {e}")
        results.append({
            "index": row.index,
            "question": row.question,
            "gold": row.gold,
            "explanation": row.explanation,
            "sufficiency": row.sufficiency,
            "is_correct_exp": row.is_correct_exp,
            "ppl_qwen": row.ppl_qwen,
            "agent_output": f"ERROR: {e}"
        })

# === 8. Save Results
results_df = pd.DataFrame(results)
logger.info("✅ Agent outputs saved to agent_outputs_sample10.csv")


2025-06-06 23:29:53,136 - INFO - Starting LangChain Agent Evaluation...
2025-06-06 23:29:53,140 - INFO - Sampled 10 low-sufficiency examples
  0%|          | 0/10 [00:00<?, ?it/s]2025-06-06 23:29:53,229 - INFO - Running agent on index: 2423
2025-06-06 23:29:54,089 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
 10%|█         | 1/10 [00:03<00:28,  3.13s/it]2025-06-06 23:29:56,362 - INFO - Running agent on index: 2143
2025-06-06 23:29:56,869 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
 20%|██        | 2/10 [00:06<00:24,  3.10s/it]2025-06-06 23:29:59,433 - INFO - Running agent on index: 2475
2025-06-06 23:29:59,877 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
 30%|███       | 3/10 [00:08<00:20,  2.88s/it]2025-06-06 23:30:02,052 - INFO - Running agent on index: 2160
2025-06-06 23:30:02,485 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/c

In [18]:
for i in results_df['agent_output']:
    print(i)

---
Question: Which of the listed quantities is measured in a unit other than the joule?
Answer: C
Redundant parts:
- "which is a unit that quantifies the total amount of visible light emitted by a source in a specific direction."
- "are used to describe the brightness or luminous flux of a light source"
- "and are distinct from the joule, which is a unit of energy."

Reasons:
- The first part is verbose; it can be simplified without losing meaning.
- The second part is implied by the context of lumens and does not need to be explicitly stated.
- The last part is unnecessary because the question already establishes the focus on units other than joules.

Concise Explanation:
"Light is measured in lumens, a unit that quantifies brightness and is different from the joule, a unit of energy."
---
Question: A student's mother is heterozygous for attached earlobes, a recessive trait. The student's father is also heterozygous for that trait. What is the percent likelihood that this student has